In [1]:
from code_tokenizer import *

code = """
if(a > 10)
{
    a = 10
}
"""

# print(color_by_char_type(code))
parsed = parse(code)

def parse_operation(value):
    operation, subject = value[1]
    parse_unknown(subject[0])
    print(operation,end=' ')
    parse_unknown(subject[1])

def parse_block(value):
    operation = value[0]
    conditon, block = value[1]
    print(operation, end=' ')
    parse_unknown(conditon)
    parse_unknown(block)

def parse_assignment(assignment):
    variable, value = assignment[1]
    parse_unknown(variable)
    print('=', end=' ')
    parse_unknown(value)


def parse_unknown(unknown):
    if unknown[0] in ['if', 'for', 'while', 'foreach']:
        parse_block(unknown)
    elif unknown[0] == 'operation':
        parse_operation(unknown)
    elif unknown[0] in ['number', 'word']:
        print(unknown[1], end=' ')
    elif unknown[0] == 'assign':
        parse_assignment(unknown)

parse_unknown(parsed)

if ( a > 10 ) { a = 10 } 